In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ase.units import Bohr
from pathlib import Path

In [ ]:
results_folder = Path("/home/moritz/SCME/scme_reference_calculations/compare_against_fortran_code/results/ice")

scme_truth = pd.read_json(
    results_folder / "scme_truth.json"
)
results = pd.read_json(
    results_folder / "results.json"
)

plot_dir = Path("./plots/ice")
plot_dir.mkdir(exist_ok=True, parents=True)

# Collect individual moments

In [ ]:
dipoles = []
quadrupoles = []
samples = []
for p in results_folder.glob("[0-9]*"):
    sample = int(p.name)
    d = np.load(p / "dipoles.npy")
    q = np.load(p / "quadrupoles.npy")
    dipoles.append(d)
    quadrupoles.append(q)
    samples.append(sample)

dipoles = np.array(dipoles)
quadrupoles = np.array(quadrupoles)

dipoles = dipoles[np.argsort(samples)]
quadrupoles = quadrupoles[np.argsort(samples)]

In [ ]:
n_water = 16

for comp in [0, 1, 2]:
    ax = plt.gca()
    ax.set_title(f"Dipole moments component {comp+1}")

    d = scme_truth["dipole"][0]
    d_truth = np.array([d[i_atom][comp] for i_atom in range(n_water)])
    d_res = dipoles[0, :n_water, comp]

    ax.plot(range(n_water), d_truth, marker="o", ms=9, label="fortran", color="C0")

    ax.plot(range(n_water), d_res, marker=".", label="moritz", color="C2")

    ax.set_ylabel(r"$\mu_{" + str(comp + 1) + "}~[e a_0]$")
    ax.set_xlabel("idx_molecule")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plot_dir / f"dms_component_{comp}.png", dpi=300)
    plt.show()
    plt.close()

In [ ]:

for comp1 in [0, 1, 2]:
    for comp2 in [0, 1, 2]:

        ax = plt.gca()
        ax.set_title(f"Quadrupole moments component {comp1+1} {comp2+1}")

        q = scme_truth["quadrupole"][0]
        q_truth = np.array([q[i_atom][comp1][comp2] for i_atom in range(n_water)])
        q_res = quadrupoles[0, :n_water, comp1, comp2]

        print(d_truth.shape)
        print(d_res.shape)


        ax.plot(
            range(n_water), q_truth, marker="o", ms=9, label="fortran", color="C0"
        )

        ax.plot(range(n_water), q_res, marker=".", label="moritz", color="C2")

        ax.set_ylabel(r"$\mu_{" + str(comp + 1) + "}~[e a_0]$")
        ax.set_xlabel("idx_molecule")
        plt.legend()
        plt.tight_layout()
        plt.savefig(plot_dir / f"qms_component_{comp1}_{comp2}.png", dpi=300)
        plt.show()
        plt.close()